## Setup

- 90% training data, 10% validation data

In [1]:
# Needed to import custom code from other directories
import sys
sys.path.append('../code')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from models import ModifiedMLPClassifier
from utils import LRAP

%matplotlib inline

SEED = 42

NUM_FEATURES = 5000
NUM_CLASSES = 3993

X_train = pd.read_csv("../data/expanded/train_features.csv", names=range(NUM_FEATURES))
y_train = pd.read_csv("../data/expanded/train_labels.csv", names=range(NUM_CLASSES))

X_valid = pd.read_csv("../data/expanded/valid_features.csv", names=range(NUM_FEATURES))
y_valid = pd.read_csv("../data/expanded/valid_labels.csv", names=range(NUM_CLASSES))

print(f"{X_train.shape[0]} examples in training set, and {X_valid.shape[0]} in validation.")

13959 examples in training set, and 1552 in validation.


## Pre-processing data

- Subtracting mean and dividing by variance.
- Statistics of training data is used for the validation data too.

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

standardize = StandardScaler()
standardize = standardize.fit(X_train)
X_train_transformed = standardize.transform(X_train)
X_valid_transformed = standardize.transform(X_valid)

# X_train_transformed = X_train
# X_valid_transformed = X_valid
# 
# normalize = MinMaxScaler()
# normalize = normalize.fit(X_train_transformed)
# X_train_transformed = normalize.transform(X_train_transformed)
# X_valid_transformed = normalize.transform(X_valid_transformed)

## Model training

While training the multi-layer perceptron, the training and validation loss are reported in each iteration (or epoch).

In [3]:
# Training takes ~10 seconds per iteration
mlp = ModifiedMLPClassifier(
    hidden_layer_sizes=(256,),
    max_iter=100,
    alpha=0.5,
    verbose=True,
    random_state=SEED,
    # custom_validation_data=(X_valid_transformed, y_valid)
)
mlp = mlp.fit(X_train_transformed, y_train)

Iteration 1, training loss = 366.20741560
Iteration 2, training loss = 50.11869569
Iteration 3, training loss = 41.02829895
Iteration 4, training loss = 36.70106804
Iteration 5, training loss = 33.92558299
Iteration 6, training loss = 31.80996072
Iteration 7, training loss = 30.05609964
Iteration 8, training loss = 28.55781174
Iteration 9, training loss = 27.23589630
Iteration 10, training loss = 26.07287757
Iteration 11, training loss = 25.02412659
Iteration 12, training loss = 24.05431919
Iteration 13, training loss = 23.18079283
Iteration 14, training loss = 22.37021015
Iteration 15, training loss = 21.62255159
Iteration 16, training loss = 20.93691998
Iteration 17, training loss = 20.29477947
Iteration 18, training loss = 19.70121674
Iteration 19, training loss = 19.15362322
Iteration 20, training loss = 18.65309881
Iteration 21, training loss = 18.18355050
Iteration 22, training loss = 17.74745905
Iteration 23, training loss = 17.35022182
Iteration 24, training loss = 16.95727908


../code\models.py:248: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


## Evaluation

In [4]:
pred_train = mlp.predict_proba(X_train_transformed)
score_train = LRAP(y_train, pred_train)
print(f"LRAP on training data: {score_train:.4f}")

LRAP on training data: 0.9251


In [5]:
pred_valid = mlp.predict_proba(X_valid_transformed)
score_valid = LRAP(y_valid, pred_valid)
print(f"LRAP on validation data: {score_valid:.4f}")

LRAP on validation data: 0.5174


In [6]:
sample_idx = 123
sample_pred = mlp.predict(X_valid_transformed[[sample_idx]])[0]
actual = y_valid.loc[sample_idx].to_numpy()

In [7]:
np.nonzero(actual)

(array([342, 418, 637, 682, 906, 992], dtype=int64),)

In [8]:
np.nonzero(sample_pred)

(array([ 128,  342,  637,  992, 1185, 1722], dtype=int64),)